In [17]:
# 引入需要的包裝

import sys
import pyvisa
import time
from PyQt6 import QtWidgets
from UI import Ui_MainWindow
#from MSO64B import MSO_64B  # 匯入 MSO_64B 類
# from AFG31101 import AFG_31101
# from RF150A100D import RF_150A100D

In [ ]:
class MSO_64B():
    def __init__(self, resource_address, visa_dll=None):
        """
        初始化儀器控制器。
        :param resource_address: 儀器的資源地址。
        :param visa_dll: 使用的 VISA DLL 路徑。如果為 None，則使用系統預設空白。
        """
        self.resource_address = resource_address
        self.visa_dll = visa_dll if visa_dll is not None else ''
        self.rm = pyvisa.ResourceManager(self.visa_dll)
        self.instrument = None
        self.connect_to_instrument()
        
    def connect_to_instrument(self):
        # 建立與儀器的連接
        try:
            self.instrument = self.rm.open_resource(self.resource_address)
            print("Connected to MSO64B.")
            print(self.query_identity())
        except Exception as e:
            print(f"Failed to connect to instrument: {e}")
            self.instrument = None
        
    def query_identity(self):
         # 查詢並返回儀器的身份識別信息
        if self.instrument:
             return self.instrument.query("*IDN?")
        else:
             raise Exception("Instrument not connected. Please connect first.")

    def close_instrument(self):
        if self.instrument:
            self.instrument.close()
            print("MSO64B connection closed.")
        else:
            print("No instrument to close.")

In [18]:
class AFG_31101():
    def __init__(self, resource_address, visa_dll=None):
        """
        初始化儀器控制器。
        :param resource_address: 儀器的資源地址。
        :param visa_dll: 使用的 VISA DLL 路徑。如果為 None，則使用系統預設空白。
        """
        self.resource_address = resource_address
        self.visa_dll = visa_dll if visa_dll is not None else ''
        self.rm = pyvisa.ResourceManager(self.visa_dll)
        self.instrument = None
        self.connect_to_instrument()
        
    def connect_to_instrument(self):
         # 建立與儀器的連接
        self.instrument = self.rm.open_resource(self.resource_address)
        print("Connected to instrument.")
        print(self.query_identity())

    def query_identity(self):
         # 查詢並返回儀器的身份識別信息
        if self.instrument:
             return self.instrument.query("*IDN?")
        else:
             raise Exception("Instrument not connected. Please connect first.")

    def close_instrument(self):
        if self.instrument:
            self.instrument.close()
            print("Instrument connection closed.")
        else:
            print("No instrument to close.")

In [19]:
class RF_150A100D():
    def __init__(self, resource_address, visa_dll=None):
        """
        初始化儀器控制器。
        :param resource_address: 儀器的資源地址。
        :param visa_dll: 使用的 VISA DLL 路徑。如果為 None，則使用系統預設空白。
        """
        self.resource_address = resource_address
        self.visa_dll = visa_dll if visa_dll is not None else ''
        self.rm = pyvisa.ResourceManager(self.visa_dll)
        self.instrument = None
        self.connect_to_instrument()
        
        
    def connect_to_instrument(self):
        # 建立與儀器的連接
        self.instrument = self.rm.open_resource(self.resource_address)
        print("Connected to instrument.")
        print(self.query_identity())

    def query_identity(self):
         # 查詢並返回儀器的身份識別信息
        if self.instrument:
             return self.instrument.query("*IDN?")
        else:
             raise Exception("Instrument not connected. Please connect first.")

    def close_instrument(self):
        if self.instrument:
            self.instrument.close()
            print("Instrument connection closed.")
        else:
            print("No instrument to close.")

In [20]:
class MainWindow(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self)
        self.mso = None  # 儀器實例變量
        self.setup_control()

    def setup_control(self):
        # 連接按鈕
        self.ui.address_refresh.clicked.connect(self.refresh_instruments)
        self.ui.connect_button.clicked.connect(self.connect_instruments)
        
    def refresh_instruments(self):
        """刷新可用的儀器地址並更新 QComboBox"""
        try:
            rm = pyvisa.ResourceManager()
            instrument_list = rm.list_resources()

            # 清空現有選項
            self.ui.Scope_address.clear()
            self.ui.FG_address.clear()
            self.ui.Amp_address.clear()

            # 添加 'none' 作為預設選項
            self.ui.Scope_address.addItem("None")
            self.ui.FG_address.addItem("None")
            self.ui.Amp_address.addItem("None")

            # 添加新選項到 ComboBox
            for address in instrument_list:
                self.ui.Scope_address.addItem(address)
                self.ui.FG_address.addItem(address)
                self.ui.Amp_address.addItem(address)

            self.statusBar().showMessage("Instrument addresses refreshed.")
        
        except Exception as e:
            self.statusBar().showMessage(f"Failed to refresh instruments: {str(e)}")

    def connect_instruments(self):
        status_message = ""  # 用來收集所有的連接狀態訊息
        
        # 連接示波器 MSO 64B
        mso_address = self.ui.Scope_address.currentText()
        if mso_address == "None" or not mso_address:
            status_message += "Failed connedct to Scope, Please select a valid instrument address.\n"
        else:
            try:
                self.mso = MSO_64B(mso_address)
                mso_idn = self.mso.query_identity()  # 查詢 IDN
                status_message += f"Scope connected to {mso_idn}.\n"
            except Exception as e:
                status_message += f"Failed to connect to Scope: {str(e)}.\n"
        
        # 連接訊號產生器 AFG 31101
        afg_address = self.ui.FG_address.currentText()
        if afg_address == "None" or not afg_address:
            status_message += "Failed connedct to Signal Generator, Please select a valid instrument address.\n"
        else:
            try:
                self.afg = AFG_31101(afg_address)
                afg_idn = self.afg.query_identity()  # 查詢 IDN
                status_message += f"Signal generator connected to {afg_idn}.\n"
            except Exception as e:
                status_message += f"Failed to connect to Signal Generator: {str(e)}.\n"
    
        # 連接功率放大器 RF150A100D
        rf_address = self.ui.Amp_address.currentText()
        if rf_address == "None" or not rf_address:
            status_message += "Failed connedct to Power Amplifier, Please select a valid instrument address.\n"
        else:
            try:
                self.rf = RF_150A100D(rf_address)
                rf_idn = self.rf.query_identity()  # 查詢 IDN
                status_message += f"Power amplifier connected to {rf_idn}.\n"
            except Exception as e:
                status_message += f"Failed to connect to Power Amplifier: {str(e)}.\n"
    
        # 最後一次性更新 statusBar
        self.statusBar().showMessage(status_message)


In [21]:
def main():
    app = QtWidgets.QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec())

if __name__ == "__main__":
    main()

Connected to instrument.
TEKTRONIX,AFG31101,C016504,SCPI:99.0 FV:1.6.1



SystemExit: 0